In [1]:
import numpy as np
import csv
from collections import defaultdict
from sklearn import linear_model
import pandas as pd

In [14]:
def csv_to_list(file_name):
    d = []
    with open(file_name, newline='') as csv_file:
        reader = csv.reader(csv_file)
        for line in reader:
            d.append({'user_id': line[0], 'recipe_id': line[1], 'date': line[2], 'rating': line[3], 'u': line[4], 'i': line[5]})
    return d[1:]

def MSE(y, ypred):
    return sum(x**2 for x in (y - ypred)) / len(y)

In [15]:
data_train = csv_to_list('interactions_train.csv')
data_valid = csv_to_list('interactions_validation.csv')
data_test = csv_to_list('interactions_test.csv')
test_Y = [float(d['rating']) for d in data_test]

In [4]:
sum([float(d['rating']) for d in data_train]) / len(data_train)

4.574089892559891

In [5]:
global_average_rating = round(sum([float(d['rating']) for d in data_train]) / len(data_train))
global_average_rating

5

In [6]:
# Logistic Regression with Steps Only

In [7]:
#PP_recipes contains information for every recipe
pp_recipes  = pd.read_csv('PP_recipes.csv')

In [8]:
pp_steps = pp_recipes['steps_tokens']
temp = pp_recipes['id']
ids = [str(num) for num in temp]
len_steps = [len(d) for d in pp_steps]

In [9]:
#Map every recipe ID to its number of steps
id_to_steps = dict(zip(ids, len_steps))

In [10]:
X_train = [[1 ,id_to_steps[d['recipe_id']]] for d in data_train]
Y_train = [float(d['rating']) for d in data_train]

In [11]:
model =linear_model.LinearRegression ( fit_intercept = False)
model.fit(X_train,Y_train)
theta = model.coef_
theta

array([4.55585841e+00, 2.64976818e-05])

In [12]:
predict_Y = [0] * len(data_test)

for i in range(len(data_test)):
    recipe = data_test[i]['recipe_id']
    if recipe in id_to_steps:
        predict_Y[i] = theta[0]+ theta[1]*id_to_steps[recipe]
    else:
        predict_Y[i] = 5


In [13]:
MSE(test_Y, np.array(predict_Y))

1.922033907295291